# Dataset Setup

IMPORTANT:

This will create more than 500 images.

It needs to be run ONCE ONLY!!!!

Unless you need a new dataset, but make sure to update the paths.

### Imports

In [2]:
import sys
import os
from config import Config
sys.path.append(Config.root_path)

import numpy as np
from Python.data_processing.image_tools import create_greyscale_masks
from Python.data_processing.image_tools import split_png_images_in_four_from_dir
from Python.data_processing.image_tools import load_image
from Python.data_processing.utils import create_weight_map
from Python.data_processing.utils import get_png_paths_from_dir
from Python.data_processing.utils import split_dataset_paths
from Python.data_processing.save_model import write_dict_to_disk

2023-04-08 11:02:43.745898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:02:44.850229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/:/home/vidarmarsh/miniconda3/envs/tf/lib/
2023-04-08 11:02:44.850366: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/lib/:/home/vidarmarsh/miniconda3/envs/tf/lib/
2023-04-08 11:02:44.850373: W tens

### Step 1
Covert masks from categorical RGB encoding to categorical greyscale encoding 

In [ ]:
# Path to directory containing RGB encoded segmentation masks
rgb_mask_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/SegmentationClass"
)
# Path to output directory for categorical greyscale segmentation masks
output_mask_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_processed/SegmentationClass_categorical"
)
# Path to json file containing class <-> RGB conversions
class_rgb_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/class_RGB.json"
)
# Path to json file containing class <-> greyscale conversions
class_greyscale_path = os.path.join(
    Config.root_path,
    r"Data/Priddy_raw/class_categorical.json"
)
create_greyscale_masks(
    rgb_mask_path, output_mask_path, class_rgb_path, class_greyscale_path
)

### Step 2
Split images into 4

In [ ]:
# Path to directory containing full size images
image_dir_path = os.path.join(
    Config.root_path, 
    r"Data/Priddy_raw/JPEGImages"
)
# Path to output directory for split images
output_dir_path = os.path.join(
    Config.root_path, 
    r"Data/Priddy_raw/JPEGImages_512"
)
split_png_images_in_four_from_dir(image_dir_path, output_dir_path)

### Step 3
Create json file with dataset filepaths partitioned in train, validation, and test, which each partition containing a fraction of the total images + segmentation masks. Create weight mask for training dataset and store in json file.

In [5]:
# Path to directory containing all 512 pixel images
image_512_dir_path = os.path.join(
    Config.root_path,
    "Data",
    "Priddy_processed",
    "JPEGImages_512"
)
# Path to directory containing all 512 pixel segmentation masks
mask_512_dir_path = os.path.join(
    Config.root_path,
    "Data",
    "Priddy_processed",
    "SegmentationClass_categorical_512"
)
# Save path for partition json file
partition_path = os.path.join(
    Config.root_path,
    "Data",
    "Priddy_processed",
    "dataset_partition_2.json"
)
# Save path for weight map json file
weight_map_path = os.path.join(
    Config.root_path,
    "Data",
    "Priddy_processed",
    "weights_2.json"
)
# Create partition
image_filenames =[
    os.path.split(path)[1] for path in get_png_paths_from_dir(image_512_dir_path)
]
mask_filenames = [
    os.path.split(path)[1] for path in get_png_paths_from_dir(mask_512_dir_path)
]

train, val, test = split_dataset_paths(
    image_filenames,
    mask_filenames,
    Config.train_size,
    Config.val_size,
    Config.test_size
)
partition = {"train": train, "validation": val, "test": test}
write_dict_to_disk(partition, partition_path)
# Create weight masks
train_mask_paths = [
    os.path.join(mask_512_dir_path, mask) for mask in train[1]
]
masks = [load_image(img) for img in train_mask_paths]
mask_dataset = np.array(masks, dtype=np.float32)
weight_mask = create_weight_map(mask_dataset)
write_dict_to_disk(weight_mask, weight_map_path)